In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
import pyspark.sql.functions as F
from pyspark.sql.functions import col, when, lit, udf
import datetime

<IPython.core.display.Javascript object>

In [2]:
%reload_kedro
%load_ext nb_black

[08/11/22 17:47:08] INFO     No path argument was provided. Using:                                    ]8;id=327348;file:///opt/homebrew/Caskroom/miniforge/base/envs/dm_project/lib/python3.10/site-packages/kedro/extras/extensions/ipython.py\ipython.py]8;;\:]8;id=6479;file:///opt/homebrew/Caskroom/miniforge/base/envs/dm_project/lib/python3.10/site-packages/kedro/extras/extensions/ipython.py#55\55]8;;\
                             /Users/juanazuero/Documents/DS/DM_project                                             

[08/11/22 17:47:08] INFO     Config from path '/Users/juanazuero/Documents/DS/DM_project/conf/local'   ]8;id=656705;file:///opt/homebrew/Caskroom/miniforge/base/envs/dm_project/lib/python3.10/site-packages/kedro/config/common.py\common.py]8;;\:]8;id=527144;file:///opt/homebrew/Caskroom/miniforge/base/envs/dm_project/lib/python3.10/site-packages/kedro/config/common.py#88\88]8;;\
                             will override the following existing top-level config keys:                           
                             spark.driver.extraJavaOptions, spark.driver.maxResultSize,                            
                             spark.driver.memory, spark.executor.extraJavaOptions,                                 
                             spark.executor.memory, spark.scheduler.mode,                                          
                             spark.sql.sources.partitionOverwriteMode                                              

                    INFO     Config from path '/Users/juanazuero/Documents/DS/DM_project/conf/local'   ]8;id=513244;file:///opt/homebrew/Caskroom/miniforge/base/envs/dm_project/lib/python3.10/site-packages/kedro/config/common.py\common.py]8;;\:]8;id=744888;file:///opt/homebrew/Caskroom/miniforge/base/envs/dm_project/lib/python3.10/site-packages/kedro/config/common.py#88\88]8;;\
                             will override the following existing top-level config keys: secop_2,                  
                             secop_2_log_in, secop_2_log_out, secop_int, secop_int_clean,                          
                             secop_int_log_in, secop_int_log_out                                                   

                    WARNING  /opt/homebrew/Caskroom/miniforge/base/envs/dm_project/lib/python3.10/s ]8;id=110345;file:///opt/homebrew/Caskroom/miniforge/base/envs/dm_project/lib/python3.10/warnings.py\warnings.py]8;;\:]8;id=338205;file:///opt/homebrew/Caskroom/miniforge/base/envs/dm_project/lib/python3.10/warnings.py#109\109]8;;\
                             ite-packages/kedro/framework/context/context.py:344: UserWarning:                     
                             Credentials not found in your Kedro project config.                                   
                             No files found in                                                                     
                             ['/Users/juanazuero/Documents/DS/DM_project/conf/base',                               
                             '/Users/juanazuero/Documents/DS/DM_project/conf/local'] matching the                  
                             glob pattern(s): ['credentials*', 'credentials*/**',                                  
                             '**/credentials*']                                                                    
                               warn(f"Credentials not found in your Kedro project                                  
                             config.\n{str(exc)}")                                                                 
                                                                                                                   

                    INFO     Kedro project secop                                                      ]8;id=727499;file:///opt/homebrew/Caskroom/miniforge/base/envs/dm_project/lib/python3.10/site-packages/kedro/extras/extensions/ipython.py\ipython.py]8;;\:]8;id=661291;file:///opt/homebrew/Caskroom/miniforge/base/envs/dm_project/lib/python3.10/site-packages/kedro/extras/extensions/ipython.py#78\78]8;;\

                    INFO     Defined global variable 'context', 'session', 'catalog' and 'pipelines'  ]8;id=918419;file:///opt/homebrew/Caskroom/miniforge/base/envs/dm_project/lib/python3.10/site-packages/kedro/extras/extensions/ipython.py\ipython.py]8;;\:]8;id=405165;file:///opt/homebrew/Caskroom/miniforge/base/envs/dm_project/lib/python3.10/site-packages/kedro/extras/extensions/ipython.py#79\79]8;;\

<IPython.core.display.Javascript object>

In [3]:
secop = catalog.load("secop_int_clean")

[08/11/22 17:47:18] INFO     Loading data from 'secop_int_clean' (SparkDataSet)...              ]8;id=515709;file:///opt/homebrew/Caskroom/miniforge/base/envs/dm_project/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=937439;file:///opt/homebrew/Caskroom/miniforge/base/envs/dm_project/lib/python3.10/site-packages/kedro/io/data_catalog.py#343\343]8;;\

<IPython.core.display.Javascript object>

In [13]:
secop.cache()

DataFrame[nivel_entidad: string, nombre_de_la_entidad: string, estado_del_proceso: string, objeto_a_contratar: string, tipo_de_contrato: string, valor_contrato: int, nom_raz_social_contratista: string, departamento_entidad: string, municipio_entidad: string, objeto_del_proceso: string, fecha_de_firma_del_contrato: date, fecha_inicio_ejecucion: date, fecha_fin_ejecucion: date, modalidad_de_contratacion: string, nit_de_la_entidad: bigint]

<IPython.core.display.Javascript object>

datetime.datetime(2022, 8, 11, 18, 0, 48, 764050)

<IPython.core.display.Javascript object>

In [10]:
secop = secop.filter(col("fecha_de_firma_del_contrato") <= datetime.datetime.today())

<IPython.core.display.Javascript object>

In [23]:
count_process_state = secop.groupBy("estado_del_proceso").count().toPandas()
count_process_state["prop"] = (
   count_process_state["count"] / count_process_state["count"].sum()
)

<IPython.core.display.Javascript object>

NaT

<IPython.core.display.Javascript object>

In [26]:
count_process_state["estado_del_proceso"].replace("", pd.NaT)

0                                       liquidado
1                                      modificado
2                               enviado proveedor
3                                          activo
4                                       terminado
5                                      prorrogado
6                                    en ejecucion
7                                      suspendido
8                                          cedido
9                                        borrador
10                                        cerrado
11                                           asdf
12                                  en aprobacion
13                         terminado sin liquidar
14                                      convocado
15                                     adjudicado
16    terminado anormalmente despues de convocado
Name: estado_del_proceso, dtype: object

<IPython.core.display.Javascript object>

In [24]:
count_process_state.sort_values("prop", ascending=False)

,estado_del_proceso,count,prop
11,celebrado,3247005,5.687293e-01
0,liquidado,1500891,2.628886e-01
6,en ejecucion,318432,5.577497e-02
1,modificado,161631,2.831048e-02
13,terminado sin liquidar,158867,2.782635e-02
3,activo,94446,1.654269e-02
14,convocado,84286,1.476312e-02
4,terminado,79528,1.392973e-02
10,cerrado,49983,8.754774e-03
15,adjudicado,7574,1.326624e-03


<IPython.core.display.Javascript object>

In [7]:
secop.select(
    F.max(col("fecha_de_firma_del_contrato")), F.min(col("fecha_de_firma_del_contrato"))
).show()

+--------------------------------+--------------------------------+
|max(fecha_de_firma_del_contrato)|min(fecha_de_firma_del_contrato)|
+--------------------------------+--------------------------------+
|                      2099-12-30|                      2000-01-01|
+--------------------------------+--------------------------------+



<IPython.core.display.Javascript object>